In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, you are also encouraged to use Piazza on Canvas.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
len(states)

57

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    6. Assign-3 university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # YOUR CODE HERE
    df = pd.read_table('6. Assign-3 university_towns.txt', header=None)
    state_dict = {}
    c_state = ""
    for k in range(len(df)):
        if '[edit]' in df.loc[k, 0]:
            c_state = df.loc[k, 0].split('[')[0]
            state_dict[c_state] = []
        else:
            c_town = df.loc[k, 0].split(' (')[0]
            state_dict[c_state].append(c_town)

    State = []
    RegionName = []
    for k in state_dict.keys():
        for v in state_dict[k]:
            State.append(k)
            RegionName.append(v)


    return pd.DataFrame(list(zip(State,RegionName)), columns=["State", "RegionName"]) 

get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [4]:
#verification code
import re
import pandas as pd
import numpy as np
# list of unique states
stateStr = """
Ohio, Kentucky, American Samoa, Nevada, Wyoming
,National, Alabama, Maryland, Alaska, Utah
,Oregon, Montana, Illinois, Tennessee, District of Columbia
,Vermont, Idaho, Arkansas, Maine, Washington
,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
,New York, Nebraska, Oklahoma, Florida, California
,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
,North Dakota, Virginia
"""
#list of regionName entries string length
regNmLenStr = """
06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
"""

df = get_list_of_university_towns()

cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
              '\s+$'))
      else 'Passed')
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
              '\('))
      else 'Passed')
print('"[" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\[')) or any(df[cols[1]].str.contains(
              '\]'))
      else 'Passed')

states_vlist = [st.strip() for st in stateStr.split(',')]

mismatchedStates = df[~df['State'].isin(
    states_vlist)].loc[:, 'State'].unique()
print('State test: ', "Passed" if len(
    mismatchedStates) == 0 else "Failed")
if len(mismatchedStates) > 0:
    print()
    print('The following states failed the equality test:')
    print()
    print('\n'.join(mismatchedStates))

df['expected_length'] = [int(s.strip())
                         for s in regNmLenStr.split(',')
                         if s.strip().isdigit()]
regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
    :, ['RegionName', 'expected_length']]
regDiff['actual_length'] = regDiff['RegionName'].str.len()
print('RegionName test: ', "Passed" if len(regDiff) ==
      0 else ' \nMismatching regionNames\n {}'.format(regDiff))

Shape test:  Passed
Index test:  Passed
Column test:  Passed
\n test:  Passed
Trailing whitespace test: Passed
"(" test: Passed
"[" test: Passed
State test:  Passed
RegionName test:  Passed


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    

    # YOUR CODE HERE
    GDP = pd.read_excel('6. Assign-3 gdplev.xls', index_col=None, header=None)
    # GDP.head()
    GDP = GDP[220:].reset_index()
    GDP = GDP[[4,6]]
    GDP.columns=['Quarter', 'GDP chained']
    
    recession_start = ''
    flag = False
    for k in range(len(GDP)-2):
        if (GDP.loc[k, 'GDP chained'] > GDP.loc[k+1, 'GDP chained']) and (GDP.loc[k+1, 'GDP chained'] > GDP.loc[k+2, 'GDP chained']):
            if flag==False:
                flag = True
                recession_start = GDP.loc[k+1, 'Quarter']
        elif (GDP.loc[k, 'GDP chained'] < GDP.loc[k+1, 'GDP chained']) and (GDP.loc[k+1, 'GDP chained'] < GDP.loc[k+2, 'GDP chained']):
            if flag==True:
                flag = False
    
    return recession_start

get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    # YOUR CODE HERE
    GDP = pd.read_excel('6. Assign-3 gdplev.xls', index_col=None, header=None)
    # GDP.head()
    GDP = GDP[220:].reset_index()
    GDP = GDP[[4,6]]
    GDP.columns=['Quarter', 'GDP chained']

    recession_end = ''
    flag = False
    for k in range(len(GDP)-2):
        if (GDP.loc[k, 'GDP chained'] > GDP.loc[k+1, 'GDP chained']) and (GDP.loc[k+1, 'GDP chained'] > GDP.loc[k+2, 'GDP chained']):
            if flag==False:
                flag = True
        elif (GDP.loc[k, 'GDP chained'] < GDP.loc[k+1, 'GDP chained']) and (GDP.loc[k+1, 'GDP chained'] < GDP.loc[k+2, 'GDP chained']):
            if flag==True:
                flag = False
                recession_end = GDP.loc[k+2, 'Quarter']
                
    return recession_end

get_recession_end()            

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # YOUR CODE HERE
    GDP = pd.read_excel('6. Assign-3 gdplev.xls', index_col=None, header=None)
    # GDP.head()
    GDP = GDP[220:].reset_index()
    GDP = GDP[[4,6]]
    GDP.columns=['Quarter', 'GDP chained']

    recession_start = get_recession_start()
    recession_end = get_recession_end()

    start_ind = GDP[GDP['Quarter']==recession_start].index.values[0]
    end_ind = GDP[GDP['Quarter']==recession_end].index.values[0]
    
    return GDP.iloc[start_ind:end_ind+1].sort_values('GDP chained').iloc[0].Quarter

get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # YOUR CODE HERE
    homes = pd.read_csv('6. Assign-3 City_Zhvi_AllHomes.csv')
    cols =['State', 'RegionName'] + list(homes.columns[-200:].values)
    homes = homes[cols]

    quarter_dict = {}
    q1 = ['01', '02', '03']
    q2 = ['04', '05', '06']
    q3 = ['07', '08', '09']
    q4 = ['10', '11', '12']

    for col in homes.columns[2:]:
        vals = col.split('-')
        if vals[1] in q1:
            m_key = vals[0]+'q1'
            if m_key in quarter_dict.keys():
                quarter_dict[m_key].append(col) 
            else:
                quarter_dict[m_key] = []
                quarter_dict[m_key].append(col) 

        elif vals[1] in q2:
            m_key = vals[0]+'q2'
            if m_key in quarter_dict.keys():
                quarter_dict[m_key].append(col) 
            else:
                quarter_dict[m_key] = []
                quarter_dict[m_key].append(col)

        elif vals[1] in q3:
            m_key = vals[0]+'q3'
            if m_key in quarter_dict.keys():
                quarter_dict[m_key].append(col) 
            else:
                quarter_dict[m_key] = []
                quarter_dict[m_key].append(col)

        elif vals[1] in q4:
            m_key = vals[0]+'q4'
            if m_key in quarter_dict.keys():
                quarter_dict[m_key].append(col) 
            else:
                quarter_dict[m_key] = []
                quarter_dict[m_key].append(col)

    for k in quarter_dict.keys():
        homes[k] = homes[quarter_dict[k]].mean(axis=1)
        
    homes['State'] = homes['State'].replace(states)

    return homes[['State', 'RegionName']+list(quarter_dict.keys())].set_index(['State', 'RegionName'])

convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [9]:
#verification code
import re
q5 = convert_housing_data_to_quarters()
pattern = re.compile("\d\d\d\dq[1-4]")
res = 'Columns type test: '
if type(q5.columns) == pd.Index: 
    res += 'Passed' 
    res += '\nColumns names test: '
    if q5.columns.map(pattern.match).astype(bool).all():
        res += 'Passed'
    else:
        res += 'Failed'
        res += "\n Do the column names look like ['2000q1','2000q2'....]"
        res += '\n PS. column names are case sensitive'
else:
    res +='Failed'
    res += "\n Did you remember to change the column names back "
    res += "to strings in the format e.g. Index(['2000q1', '2000q2',...],dtype='object')"
res += "\nState name test: "    
if set(q5.index.get_level_values(0)).issubset(set(states.values())):
    res += 'Passed' 
else:
    res += 'Failed'
    res += "\n Did you map the abbreviated state names to the full names using the precoded states dictionary"
res += "\nShape test: "
if q5.shape == (10730,67):
    res += 'Passed'
else:
    res += 'Failed'

#generate all column names
columns = set(pd.date_range(
    start="2000", end="2017", freq='Q').map(
    lambda x: "{:}q{:}".format(x.year,x.quarter))[:-1])
res += "\nColumn Names test: "
if set(q5.columns)==columns:
    res += 'Passed'
else:
    res += 'Failed'
    res += "\nmissing columns" + str(columns-set(q5.columns))

print(res)

Columns type test: Passed
Columns names test: Passed
State name test: Passed
Shape test: Passed
Column Names test: Passed


In [10]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # YOUR CODE HERE
    columns = list(pd.date_range(
    start="2000", end="2017", freq='Q').map(
    lambda x: "{:}q{:}".format(x.year,x.quarter))[:-1])
    
    homes = convert_housing_data_to_quarters()
    uni_towns = get_list_of_university_towns()

    recession_start = get_recession_start()
    recession_end = get_recession_end()
    recession_bottom = get_recession_bottom()
    quarter_before_recession = columns[columns.index(recession_start)-1]


    homes['price_ratio'] = homes[quarter_before_recession]/homes[recession_bottom]

    uni_towns['University Town'] = 1

    homes = homes.join(uni_towns.set_index(['State', 'RegionName']))

    u_towns_price = homes[homes['University Town']==1]['price_ratio']
    non_u_towns_price = homes[homes['University Town'].isnull()]['price_ratio']

    p = ttest_ind(u_towns_price.values, non_u_towns_price.values, nan_policy='omit').pvalue

    university_towns_ratio_mean = homes[homes['University Town']==1]['price_ratio'].mean()
    non_university_towns_ratio_mean = homes[homes['University Town'].isnull()]['price_ratio'].mean()

    better = 'non-university town'
    if university_towns_ratio_mean < non_university_towns_ratio_mean:
        better = 'university town'

    different = True if p<0.01 else False

    return (different, p, better)

run_ttest()

(True, 0.002724063704753125, 'university town')

<div align=right>Acknowledgement of Credit: Dr. Christopher Brooks and Sophia, University of Michigan</div>